In [ ]:
import hoomd
import hoomd.md
import hoomd.metal
from hoomd import deprecated
import mbuild, mdtraj, nglview, numpy
import matplotlib.pyplot as plt
%matplotlib inline

import fresnel
import gsd
import gsd.fl
import gsd.hoomd
import numpy
import PIL
import IPython
import io
import math
import sys

device = fresnel.Device(mode='cpu');
preview_tracer = fresnel.tracer.Preview(device, 300, 300, aa_level=3)
path_tracer = fresnel.tracer.Path(device, 300, 300)

blue = fresnel.color.linear([0.25,0.5,1])*0.9;
orange = fresnel.color.linear([1.0,0.714,0.169])*0.9

def render_sphere_frame(frame, height=None):

    if height is None:
        if hasattr(frame, 'configuration'):
            Ly = frame.configuration.box[1]
            height = Ly * math.sqrt(3)
        else:
            Ly = frame.box.Ly;
            height = Ly * math.sqrt(3)

    scene = fresnel.Scene(device)
    scene.lights = fresnel.light.cloudy();
    g = fresnel.geometry.Sphere(scene, position=frame.particles.position, radius=numpy.ones(frame.particles.N)*0.5)
    g.material = fresnel.material.Material(solid=0.0, color=blue, primitive_color_mix=1.0, specular=1.0, roughness=0.2)
    g.outline_width = 0.07
    scene.camera = fresnel.camera.orthographic(position=(height, height, height), look_at=(0,0,0), up=(0,1,0), height=height)

    g.color[frame.particles.typeid == 0] = blue;
    g.color[frame.particles.typeid == 1] = orange;

    scene.background_color = (1,1,1)

    return path_tracer.sample(scene, samples=64, light_samples=20)

def display_movie(frame_gen, gsd_file):
    f = gsd.fl.GSDFile(gsd_file, 'rb')
    t = gsd.hoomd.HOOMDTrajectory(f)

    a = frame_gen(t[0]);

    if tuple(map(int, (PIL.__version__.split(".")))) < (3,4,0):
        print("Warning! Movie display output requires pillow 3.4.0 or newer.")
        print("Older versions of pillow may only display the first frame.")

    im0 = PIL.Image.fromarray(a[:,:, 0:3], mode='RGB').convert("P", palette=PIL.Image.ADAPTIVE);
    ims = [];
    for f in t[1:]:
        a = frame_gen(f);
        im = PIL.Image.fromarray(a[:,:, 0:3], mode='RGB')
        im_p = im.quantize(palette=im0);
        ims.append(im_p)

    f = io.BytesIO()
    im0.save(f, 'gif', save_all=True, append_images=ims, duration=1000, loop=0)

    if (sys.version_info[0] >= 3):
        size = len(f.getbuffer())/1024;
        if (size > 2000):
            print("Size:", size, "KiB")
    return IPython.display.display(IPython.display.Image(data=f.getvalue()))

def run_pure_Fe(kT):
    Temperature = kT #Note, with EAM, temperature is measured in electron-Volts
    hoomd.context.initialize("")
    system=hoomd.init.create_lattice(unitcell=hoomd.lattice.fcc(a=2.,type_name="Fe"),n=6)
    deprecated.dump.xml(filename="Fe-init.hoomdxml",group=hoomd.group.all(),vis=True)
    hoomd.md.integrate.mode_standard(dt=0.001)
    nl = hoomd.md.nlist.cell()
    eam = hoomd.metal.pair.eam(file='MCM2011_eam.fs', type='FS', nlist=nl)
    all=hoomd.group.all()
    nvt=hoomd.md.integrate.nvt(group=all,kT=Temperature,tau=0.1)
    nvt.randomize_velocities(seed=23) # Randomizing velocities speeds up equilibration.
    hoomd.dump.gsd("Fe.gsd", period=10, group=all, overwrite=True) # write out configurations periodically. Can skip if doing all analysis in real-time
    hoomd.analyze.log(filename="Fe.log",quantities=['potential_energy','temperature','pressure','kinetic_energy'],period=5,overwrite=True)
    hoomd.run(100)
    deprecated.dump.xml(filename="Fe-final.hoomdxml",group=hoomd.group.all(),vis=True)
    
def run_alloy(kT):
    Temperature = kT #Note, with EAM, temperature is measured in electron-Volts
    hoomd.context.initialize("")
    unit = hoomd.lattice.unitcell(N=2,a1=[4.0,0,0],a2=[0,4.0,0],a3=[0,0,4.],type_name=["Ni","Cu"],position=[[0,0,0],[2.0,0,0]])
    system=hoomd.init.create_lattice(unitcell=unit,n=6)
    deprecated.dump.xml(filename="Alloy-init.hoomdxml",group=hoomd.group.all(),vis=True)
    hoomd.md.integrate.mode_standard(dt=0.001)
    nl = hoomd.md.nlist.cell()
    eam = hoomd.metal.pair.eam(file='CuNi_v2.eam.alloy', type='Alloy', nlist=nl)
    all=hoomd.group.all()
    nvt=hoomd.md.integrate.nvt(group=all,kT=Temperature,tau=0.1)
    nvt.randomize_velocities(seed=23) # Randomizing velocities speeds up equilibration.
    hoomd.dump.gsd("Alloy.gsd", period=10, group=all, overwrite=True) # write out configurations periodically. Can skip if doing all analysis in real-time
    hoomd.analyze.log(filename="Alloy.log",quantities=['potential_energy','temperature','pressure','kinetic_energy'],period=5,overwrite=True)
    hoomd.run(100)
    deprecated.dump.xml(filename="Alloy-final.hoomdxml",group=hoomd.group.all(),vis=True)
    

In [ ]:
#Pure Fe
run_pure_Fe(20.0) 
display_movie(render_sphere_frame, 'Fe.gsd');


In [ ]:
#Ni Cu Alloy
run_alloy(10.0)
display_movie(render_sphere_frame, 'Alloy.gsd');

In [ ]:
#Ooh fancy; we don't need to open up that terminal after all!
!rm -rf diffract* # remove old diffraction data
#!diffract -ftype hoomdxml -tp Alloy-init.hoomdxml
#!diffract -ftype hoomdxml -tp Alloy-final.hoomdxml
!diffract -ftype hoomdxml -tp Alloy-init.hoomdxml -ftype gsd -tj Alloy.gsd --all
